In [1]:
# Makes the display take up more of the screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
# Export
import os
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [3]:
# Export
from nb_files.nb_NeuralNetwork import ModelMaker

In [4]:
# Export
class DQN(nn.Module):
    def __init__(self, lr, n_actions, name, input_dims, chkpt_dir, dropout):
        super(DQN, self).__init__()
        self.checkpoint_dir = chkpt_dir
        self.checkpoint_file = os.path.join(self.checkpoint_dir, name)
        self.dropout = dropout
        self.model = ModelMaker(arch='tf_mobilenetv3_small_075',
                                input_channels=input_dims[0],
                                num_outputs=n_actions, dropout=self.dropout)        

        self.optimizer = optim.Adam(self.parameters(), lr=lr, weight_decay=0.01)
        self.loss = nn.MSELoss()
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)

    def forward(self, state):        
        actions = self.model(state)
        return actions

    def save_checkpoint(self):
        print('... saving checkpoint ...')
        T.save(self.state_dict(), self.checkpoint_file)

    def load_checkpoint(self):
        print('... loading checkpoint ...')
        self.load_state_dict(T.load(self.checkpoint_file))

In [5]:
Q=DQN(lr=0.001, n_actions=7,  input_dims=(4, 84, 84),
        name='Test', chkpt_dir='models/', dropout=0.001)


In [6]:
Q.model

Sequential(
  (0): Sequential(
    (0): Conv2d(4, 16, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (1): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (2): Hardswish()
    (3): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2dSame(16, 16, kernel_size=(3, 3), stride=(2, 2), groups=16, bias=False)
          (bn1): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): ReLU(inplace=True)
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): ReLU(inplace=True)
            (conv_expand): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
            (gate): Hardsigmoid()
          )
          (conv_pw): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act2): Identity()

In [7]:
!python notebook2script.py DQN.ipynb

Converted DQN.ipynb to nb_files/nb_DQN.py


In [8]:
import datetime as dt
end=dt.datetime.now()
print(f'Finished: {end.strftime("%A %B %d, %Y")} at {end.strftime("%H:%M")}')

Finished: Sunday February 20, 2022 at 10:28
